<a href="https://colab.research.google.com/github/sdgroeve/Machine-Learning-Course-2days/blob/main/mnist-deep-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False, cache=False)

In [2]:
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

X /= 255

In [3]:
# Split the data into training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [4]:
import torch
from torch import nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
import numpy as np

mnist_dim = X.shape[1]
hidden_dim = int(mnist_dim/8)
output_dim = len(np.unique(mnist.target))

In [7]:
mnist_dim, hidden_dim, output_dim

(784, 98, 10)

In [8]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            input_dim=mnist_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

In [10]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 185 kB 14.1 MB/s 


In [11]:
from skorch import NeuralNetClassifier

In [12]:
torch.manual_seed(0)

net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device=device,
)

In [13]:
net.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7541       0.9033        0.3506  3.4516
      2        0.4041       0.9218        0.2732  1.1447
      3        0.3367       0.9332        0.2362  1.1315
      4        0.3015       0.9405        0.2089  1.1368
      5        0.2732       0.9446        0.1905  1.1416
      6        0.2544       0.9489        0.1786  1.1011
      7        0.2431       0.9523        0.1683  1.1324
      8        0.2298       0.9547        0.1585  1.1253
      9        0.2171       0.9562        0.1527  1.1519
     10        0.2106       0.9572        0.1464  1.1333
     11        0.2031       0.9608        0.1394  1.1227
     12        0.1966       0.9623        0.1355  1.1373
     13        0.1957       0.9617        0.1324  1.1579
     14        0.1897       0.9634        0.1294  1.1523
     15        0.1815       0.9638        0.1258  1.1353
     16        0.1783       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dropout): Dropout(p=0.5, inplace=False)
    (hidden): Linear(in_features=784, out_features=98, bias=True)
    (output): Linear(in_features=98, out_features=10, bias=True)
  ),
)

In [14]:
y_predicted = net.predict(X_test)
# Calculate the accuracy of the prediction
from sklearn.metrics import accuracy_score
print("Accuracy = {} %".format(accuracy_score(y_test, y_predicted)*100))
# Cross validate the scores
from sklearn.metrics import classification_report
print("Classification Report \n {}".format(classification_report(y_test, y_predicted, labels=range(0,10))))

Accuracy = 96.48571428571428 %
Classification Report 
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       346
           1       0.97      0.98      0.98       378
           2       0.94      0.97      0.95       332
           3       0.97      0.95      0.96       400
           4       0.96      0.97      0.96       295
           5       0.96      0.98      0.97       335
           6       0.96      0.98      0.97       351
           7       0.97      0.96      0.96       362
           8       0.96      0.95      0.96       348
           9       0.97      0.95      0.96       353

    accuracy                           0.96      3500
   macro avg       0.96      0.97      0.96      3500
weighted avg       0.96      0.96      0.96      3500



In [ ]:
from sklearn.model_selection import GridSearchCV

idx = np.random.choice(X_train.shape[0], 10000, replace=False)
X_train_small = X_train[idx]
y_train_small = y_train[idx]

In [ ]:
grid = {
    "C":np.logspace(-3,3,7)
}

clf_cv = GridSearchCV(clf, grid, cv=5, verbose=2)

In [ ]:
grid

{'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])}

In [ ]:
clf_cv.fit(X_train_small,y_train_small)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV] END ............................................C=0.001; total time=   2.5s
[CV] END ............................................C=0.001; total time=   2.8s
[CV] END ............................................C=0.001; total time=   2.4s
[CV] END ............................................C=0.001; total time=   2.6s
[CV] END ............................................C=0.001; total time=   2.8s
[CV] END .............................................C=0.01; total time=   4.9s
[CV] END .............................................C=0.01; total time=   5.0s
[CV] END .............................................C=0.01; total time=   5.0s
[CV] END .............................................C=0.01; total time=   5.0s
[CV] END .............................................C=0.01; total time=   4.9s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ..............................................C=0.1; total time=   6.8s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ..............................................C=0.1; total time=   7.0s


...

[CV] END ...........................................C=1000.0; total time=   7.0s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ...........................................C=1000.0; total time=   7.1s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=200),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])},
             verbose=2)

In [ ]:
import pandas as pd

result_cv = pd.DataFrame()
result_cv["param_C"] = clf_cv.cv_results_["param_C"].data
result_cv["score"] = clf_cv.cv_results_["mean_test_score"]

result_cv

,param_C,score
0,0.001,0.8565
1,0.01,0.8925
2,0.1,0.9068
3,1.0,0.8973
4,10.0,0.8759
5,100.0,0.8675
6,1000.0,0.8634


In [ ]:
clf = LogisticRegression(C=0.1, max_iter=200)

# Perform the predictions
clf.fit(X_train,y_train)
# Perform the predictions
y_predicted = clf.predict(X_test)
# Calculate the accuracy of the prediction
from sklearn.metrics import accuracy_score
print("Accuracy = {} %".format(accuracy_score(y_test, y_predicted)*100))
# Cross validate the scores
from sklearn.metrics import classification_report
print("Classification Report \n {}".format(classification_report(y_test, y_predicted, labels=range(0,10))))

Accuracy = 91.91428571428571 %
Classification Report 
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       346
           1       0.94      0.98      0.96       378
           2       0.87      0.90      0.88       332
           3       0.91      0.89      0.90       400
           4       0.92      0.94      0.93       295
           5       0.88      0.88      0.88       335
           6       0.94      0.94      0.94       351
           7       0.92      0.93      0.92       362
           8       0.90      0.87      0.89       348
           9       0.94      0.90      0.92       353

    accuracy                           0.92      3500
   macro avg       0.92      0.92      0.92      3500
weighted avg       0.92      0.92      0.92      3500



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
